In [1]:
import torch
from torch import nn
from d2l import torch as d2l

dropout=0.5
net= nn.Sequential(
    nn.Conv2d(1,6,kernel_size=5,padding=2),nn.ReLU(),
    nn.AvgPool2d(kernel_size=2,stride=2),
    nn.Conv2d(6,16,kernel_size=5),nn.ReLU(),
    nn.AvgPool2d(kernel_size=2,stride=2),
    nn.Flatten(),
    nn.Linear(16*5*5,120),nn.ReLU(),
    nn.Linear(120,84),nn.Sigmoid(),
    nn.Linear(84,10)
)

In [2]:
batch_size=256
train_iter,test_iter=d2l.load_data_fashion_mnist(batch_size=batch_size)


In [3]:
def evaluate_accuracy_gpu(net,data_iter,device=None):
    if isinstance(net,nn.Module):
        net.eval()#设置为评估模式
        if not device:
            device=next(iter(net.parameters())).device
    
    metric=d2l.Accumulator(2)
    with torch.no_grad():
        for X,y in data_iter:
            if isinstance(X,list):
                X= [x.to(device) for x in X]
            else:
                X = X.to(device)
            y = y.to(device)
            metric.add(d2l.accuracy(net(X),y),y.numel()) #numel元素数量
    return metric[0]/metric[1]

In [4]:
def train(net,train_iter,test_iter,num_epochs,lr,device):
    #用gpu训练模型
    def init_weights(m):
        if type(m)== nn.Linear or type(m)== nn.Conv2d:
            nn.init.xavier_uniform_(m.weight)
    net.apply(init_weights)
    print('training on', device)
    net.to(device)
    optimizer = torch.optim.SGD(net.parameters(),lr=lr)
    loss=nn.CrossEntropyLoss()
    
    timer,num_batches=d2l.Timer(),len(train_iter)   #计时器
    
    for epochs in range(num_epochs):
        metric=d2l.Accumulator(3)
        net.train()
        for i,(X,y) in enumerate(train_iter):##获得索引值
            timer.start()#开始计时
            optimizer.zero_grad()
            X,y=X.to(device),y.to(device)
            y_hat=net(X)
            l=loss(y_hat,y)
            l.backward()
            optimizer.step()#通过梯度下降优化
            with torch.no_grad():
                metric.add(l*X.shape[0],d2l.accuracy(y_hat,y),X.shape[0])
                
            timer.stop()#结束计时  计算梯度下降的时间
            
            train_l=metric[0]/metric[2]
            train_acc=metric[1]/metric[2]
        test_acc=evaluate_accuracy_gpu(net,test_iter)
    print(f'loss {train_l:.3f}, train acc {train_acc:.3f}, '
          f'test acc {test_acc:.3f}')
    print(f'{metric[2] * num_epochs / timer.sum():.1f} examples/sec '
          f'on {str(device)}')
    
    
    

In [5]:
lr,num_epochs=0.01,10
train(net,train_iter,test_iter,num_epochs,lr,d2l.try_gpu())

training on cuda:0
loss 0.715, train acc 0.742, test acc 0.745
45417.4 examples/sec on cuda:0
